In [1]:
from Utils.scrapping_file import Scraping_Class
from Utils.constants import *

import pandas as pd
import numpy as np

In [2]:
scraping_class = Scraping_Class()
price_steps = [0,50,100,150,200,250,300,500,700,1000,1500,2000,3000]

In [3]:
final_df = pd.DataFrame(columns=['name','SKU','price','department','sector'])


for department in list(DEPARTMENT_SECTOR.keys()):
    print(f'===========================================================|d|\nscaning department: {department}')

    for sector in (DEPARTMENT_SECTOR[department]):
        print(f'===============================================|s|\nscaning sector: {sector}')
        all_links = []
        
        for n in range(len(price_steps)):
            if n+1<len(price_steps):
                min_price = price_steps[n]
                max_price = price_steps[n+1]

            print(f'=========|p|\nscaning prices: {min_price} to {max_price}')

            url = scraping_class.URL_TEMPLATE(department=department, sector=sector, page_num=1, min_price=min_price, max_price=max_price)

            num_products = scraping_class.scan_num_products(url)
            pages_num = int(np.ceil(num_products/28))
            if pages_num > 50:
                pages_num=50

            if pages_num>0:    
                for page in range(pages_num):
                    page+=1
                    print(f'scaning page: {page} of {pages_num}')
                    url = scraping_class.URL_TEMPLATE(department=department, sector=sector, page_num=page, min_price=min_price, max_price=max_price)
                
                    links = scraping_class.scan_products(url)

                    all_links.extend(links)

        priceList = []
        nameList = []
        skuList = []
        for product in all_links:
            price,name,sku = scraping_class.get_product_info('https://www.rihappy.com.br'+product)
            priceList.append(price)
            nameList.append(name)
            skuList.append(sku)
        
        temp_df = pd.DataFrame({'name':nameList,'SKU':skuList,'price':priceList,'department':[department]*len(all_links),'sector':[sector]*len(all_links)})
        final_df = pd.concat([final_df,temp_df]).reset_index(drop=True)

        


===========================================================|d|
scaning department: brinquedos
===============================================|s|
scaning sector: jogos
=========|p|
scaning prices: 0 to 50
scaning page: 1 of 50
scaning page: 2 of 50
scaning page: 3 of 50
scaning page: 4 of 50
scaning page: 5 of 50
scaning page: 6 of 50
scaning page: 7 of 50
scaning page: 8 of 50
scaning page: 9 of 50
scaning page: 10 of 50
scaning page: 11 of 50
scaning page: 12 of 50
scaning page: 13 of 50
scaning page: 14 of 50
scaning page: 15 of 50
scaning page: 16 of 50
scaning page: 17 of 50
scaning page: 18 of 50
scaning page: 19 of 50
scaning page: 20 of 50
scaning page: 21 of 50
scaning page: 22 of 50
scaning page: 23 of 50
scaning page: 24 of 50
scaning page: 25 of 50
scaning page: 26 of 50
scaning page: 27 of 50
scaning page: 28 of 50
scaning page: 29 of 50
scaning page: 30 of 50
scaning page: 31 of 50
scaning page: 32 of 50
scaning page: 33 of 50
scaning page: 34 of 50
scaning page: 35 of 50


KeyboardInterrupt: 

In [ ]:
final_df

,name,SKU,price,department,sector
0,Fralda de Pano - Cremer - Luxo com Bainha - Br...,100052151,36.90,baby,banho-e-higiene
1,Fraldas Pampers Recém-Nascido Premium Care RN+...,1002529183,25.90,baby,banho-e-higiene
2,Livrinho de Banho - Conhecendo as Frutas - Buba,100287566,17.90,baby,banho-e-higiene
3,Assento para Banheira - Burigotto - Cinza,1002522750,49.99,baby,banho-e-higiene
4,Massageador de Gengiva - Com Case - Buba,100160036,18.90,baby,banho-e-higiene
...,...,...,...,...,...
2172,Berço Co-Sleeper Maxi-Cosi Iora Air Beyond Gra...,1002659366,2199.90,baby,sono-do-bebe
2173,Berço Portátil Maxi-Cosi Iora Air Beyond Graph...,1002694405,2269.98,baby,sono-do-bebe
2174,Berço Multifuncional Casinha Alicia com Colchã...,1002640138,2229.90,baby,sono-do-bebe
2175,Berço Co-Sleeper Maxi-Cosi Iora Air Beyond Gra...,1002659366,2199.90,baby,sono-do-bebe


In [4]:
final_df.to_csv("ri_happy_4.csv",sep=';', encoding='cp1252')

In [9]:
df0 = pd.read_csv('ri_happy_0.csv',sep=',')
df1 = pd.read_csv('ri_happy_1.csv',sep=';',  encoding='cp1252')
df2 = pd.read_csv('ri_happy_2.csv',sep=';',  encoding='cp1252')
df3 = pd.read_csv('ri_happy_3.csv',sep=';',  encoding='cp1252')

final_df = pd.concat([df0,df1,df2,df3],ignore_index=True).reset_index(drop=True)


In [12]:
final_df.drop('Unnamed: 0',axis=1).to_csv("ri_happy.csv",sep=';', encoding='cp1252')